In [177]:
#References
#https://bootcampspot-v2.com/sessions/472909/videos/25279
#http://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.set_value.html
#https://stackoverflow.com/questions/5244810/python-appending-a-dictionary-to-a-list-i-see-a-pointer-like-behavior
#https://stackoverflow.com/questions/5242933/what-was-the-motivation-for-doing-lists-augmented-assignment-in-place-in-py
#https://stackoverflow.com/questions/4432208/how-does-work-in-python
#https://python-reference.readthedocs.io/en/latest/docs/operators/modulus.html
#https://stackoverflow.com/questions/43668827/python-modulo-function
#https://stackoverflow.com/questions/44513738/pandas-create-empty-dataframe-with-only-column-names
#https://www.tutorialspoint.com/python/string_title.htm
#https://stackoverflow.com/questions/134934/display-number-with-leading-zeros
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"


In [5]:
# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [153]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

In [176]:
#using the starter code, build a dataframe for lat/long visaid
#maybe create a dataframe with empty columns and then add to it?
#https://stackoverflow.com/questions/44513738/pandas-create-empty-dataframe-with-only-column-names

#bad syntax missing {}
#latlong_df = pd.DataFrame("Lat": lats, "Long": lngs)

latlong_df = pd.DataFrame({"Lat": lats, "Long": lngs})

latlong_df.head()

,Lat,Long
0,79.683488,34.973856
1,-21.774200,-129.747536
2,34.918684,74.971213
3,10.995184,23.468342
4,59.184856,-123.048729


In [155]:
lat_lngs = zip(lats, lngs)

citiesnearby = []

In [156]:

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #find the total amount of cities nearby because im curious
    citiesnearby.append(city)
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        #print(city)
        cities.append(city)

# Print the city count to confirm sufficient count
#print(len(cities))
#print(len(citiesnearby))

num_cities_found = len(cities)
num_coordinates = len(citiesnearby)

print(f'Using {num_coordinates} coordinates, found {num_cities_found} citie(s)')


Using 1500 coordinates, found 602 citie(s)


In [157]:
cities_df = pd.DataFrame({"Cities":cities})
cities_df.head()

,Cities
0,vardo
1,rikitea
2,bandipur
3,birao
4,fort nelson


In [158]:
baseurl = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
units = "imperial"

# Build partial query URL
query_url = f"{baseurl}appid={api_key}&units={units}&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=69958e30913d3cf9ab8231330ad52ba7&units=imperial&q=


In [188]:

#used for testing only --------->
#track the unique responses, found 2 [200, 404]
#responsetypes = [] 
#<-------------------------------

#set up a counter
counter = 0
numbreak = 25

#count cities added
citiesadded = 0

#set up a grouping or modulus group using %
modnum = 5
modgroup = 0

#weathercities = {"city":,"country":}
#dictionary object not working using list instead
weathercities = []

#testing print --------------------------->
print(f'Begin Processing from {len(cities)} Cities. {numbreak} Attempts')
print('Beginning Data Retrieval')
print('-----------------------------')

#<-----------end testing print


#loop through the dataframe
for index, row in cities_df.iterrows():
    #print(row["Cities"])
    thecity = row["Cities"]
    city_url = query_url + thecity
    #print(city_url)
    
    city_response = requests.get(city_url).json()
        

    ##This section for testing only ---------->
    ##checking for bad responses, only check to see if its possible to get have errors
    ##results are we found 2 different response types so we have to deal with them
    #responsetype = requests.get(city_url).status_code
    #if responsetype not in responsetypes:
    #    responsetypes.append(responsetype)
    ##<-----------------End testing reponse types
    
    #testing mod groups
    #if (index % 2 == 0):
        #modgroup +=1
    #<----end mod testing
        
    
    
    #if we find a response with 404 we need to skip it
    responsetype = requests.get(city_url).status_code
    #print(responsetype)
    
    if responsetype == 200:
        citiesadded += 1
        #print(responsetype)
        #print(city_response["id"])
        #print(city_response["sys"]["country"])
        city_id = city_response["id"]
        city_country = city_response["sys"]["country"]
        city_lat = city_response["coord"]["lon"]
        city_long = city_response["coord"]["lat"]
        city_name = city_response["name"]
        city_temp_max = city_response["main"]["temp_max"]
        city_humidity = city_response["main"]["humidity"]
        city_clouds = city_response["clouds"]["all"]
        city_winds = city_response["wind"]["speed"]
        
        #make it so that the cities added Groupings are ONLY for cities added, 
        #and NOT for the entire list we are searching, this would give us inconsistencies in groups
        if (citiesadded % modnum ==0):
            modgroup +=1
        
        #print(f'ID:{city_id} Country:{city_country} City:{city_name}')
        

        weathercities.append({"cityid":city_id
                              ,"country":city_country
                              ,"city":city_name
                              ,"lat":city_lat
                              ,"long":city_long
                              ,"max temp":city_temp_max
                              ,"humidity":city_humidity
                              ,"cloudiness":city_clouds
                              ,"wind speed":city_winds
                              ,"grouping":modgroup
                             })

        #print(f'Adding {counter} city{city_name} ModGroup# {modgroup}')
        #"{:10.4f}".format(x)
        #https://stackoverflow.com/questions/134934/display-number-with-leading-zeros
        print(f'{str(index).zfill(2)}: Add  {citiesadded} City: {thecity.title()} Group {modgroup}')
        
    else:
        #print(f'City:{thecity} Response:{responsetype}')
        print(f'{str(index).zfill(2)}: Skip ** City:{thecity.title()} not found.')
    
    #print(f'ModGroup# {modgroup}')
    
    
    #testing for the first 5 records
    if counter == numbreak:
        break
    
      
    counter+=1
        
        
print('-----------------------------')
print('Data Retrieval Complete      \n')
print('********SUMMARY**************')
print(f'Using {num_coordinates} coordinates, found {num_cities_found} citie(s).')
print(f'Created {modgroup} group(s) from a total of {len(weathercities)} citie(s).')
print('-----------------------------')


Begin Processing from 602 Cities. 25 Attempts
Beginning Data Retrieval
-----------------------------
00: Add  1 City: Vardo Group 0
01: Add  2 City: Rikitea Group 0
02: Add  3 City: Bandipur Group 0
03: Add  4 City: Birao Group 0
04: Add  5 City: Fort Nelson Group 1
05: Add  6 City: Hirara Group 1
06: Add  7 City: Kalemie Group 1
07: Add  8 City: Barrow Group 1
08: Add  9 City: Iqaluit Group 1
09: Add  10 City: Brownsville Group 2
10: Add  11 City: San Cristobal Group 2
11: Add  12 City: Ilulissat Group 2
12: Skip ** City:Barentsburg not found.
13: Skip ** City:Vaitupu not found.
14: Add  13 City: Ushuaia Group 2
15: Add  14 City: Teknaf Group 2
16: Add  15 City: Cherskiy Group 3
17: Add  16 City: Provideniya Group 3
18: Add  17 City: Nador Group 3
19: Add  18 City: Karabash Group 3
20: Add  19 City: Saint-Paul Group 3
21: Add  20 City: Kapaa Group 4
22: Add  21 City: Karratha Group 4
23: Add  22 City: Vanimo Group 4
24: Add  23 City: Tasiilaq Group 4
25: Add  24 City: Beringovskiy Gro

In [189]:
print(weather_cities_df.count())
#weathercities[0:2]
weathercities[0]

city          24
cityid        24
couldiness    24
country       24
grouping      24
humidity      24
lat           24
long          24
max temp      24
wind speed    24
dtype: int64


{'cityid': 4372777,
 'country': 'US',
 'city': 'Vardo',
 'lat': -77.74,
 'long': 39.62,
 'max temp': 95,
 'humidity': 44,
 'cloudiness': 40,
 'wind speed': 6.93,
 'grouping': 0}

In [190]:
weather_cities_df = pd.DataFrame(weathercities)
weather_cities_df.head()

,city,cityid,cloudiness,country,grouping,humidity,lat,long,max temp,wind speed
0,Vardo,4372777,40,US,0,44,-77.74,39.62,95.00,6.93
1,Rikitea,4030556,64,PF,0,100,-134.97,-23.12,71.14,28.32
2,Bandipur,1270686,88,IN,0,95,76.63,11.67,64.39,2.04
3,Birao,240210,44,CF,0,98,22.78,10.30,70.42,4.05
4,Fort Nelson,5955902,75,CA,1,61,-122.69,58.81,46.40,3.36


In [183]:
#export the dataframe to csv
weather_cities_df.to_csv("../data/testing_cities.csv")

In [187]:
#get the plot values

#x plot lattitudes
xplot_lats = weather_cities_df["lat"]

#temperatures
yplot_temp = weather_cities_df["max temp"]

#humidities
yplot_humid = weather_cities_df["humidity"]

#cloudiness
yplot_clouds = weather_cities_df[""]

#wind speeds
yplot_winds = weather_cities_df["max temp"]


In [71]:
#city_data = {"id","lat","lng"}
city_data = pd.DataFrame([
                            {
                                "id"
                                ,"Country"
                            }
    
])


In [62]:
city_data.append({"id":4,"lat","lng"})
#city_data.append({"id":5,"lat":6,"lng"})
#city_data.append({"id":6,"lat":7,"lng":8})

SyntaxError: invalid syntax (<ipython-input-62-bca96e453886>, line 1)

In [60]:
city_data

[{'id', 'lat', 'lng'}]